## Managing the OpenAPI Session

After login, the client will automatically retrieve a token pair (access and refresh token) from the endpoint specified in your app's application config. This is a default `/token` endpoint specific to the environment that your app is running in, which is what all applications use to generate tokens (standard OAuth implementation).

At this point, a *OpenAPI session* has commenced, identified by a session id which typically looks something like this:

```
95af38e86c1547fdafc4204a0684ded7
```

By default, the access token is only valid for 20 minutes, after which it expires and the session is automatically disconnected. Requests to OpenAPI will error out with a 401 Unauthorized response if an expired token is used, which effectively means that the client is no longer logged in. Restricting token lifetimes is a fairly typical security measure to prevent tokens 'falling in the wrong hands' from being useful for extended periods (reducing the attack surface of a potential bad actor).

The refresh token's lifetime is typically 1 hour, which means that at any time in the hour starting from the moment the tokens are generated, the session can be extended by exercising the refresh token. This is what is typically referred to as 'keeping the session alive'.

The client implements a basic `refresh()` method that does exactly this - it makes a request to the token endpoint and obtains a fresh token pair, which replaces the existing tokens saved in the client's state (keeping the session 'alive'). These tokens in turn last for 20 and 60 minutes respectively and are used for subsequent requests.

Therefore, it is up to the client to systematically refresh the token pair _before_ the access token expires - otherwise requests will start failing.

## In Jupyter Notebooks

Because of the ease-of-use and the prevalence of Jupyter Notebooks in the Python community, the client has a simple feature to keep the session authenticated as long as the notebook is 'running' (i.e. it is not closed/exited and the kernel is not restarted).

When the `login()` method is called, set the following parameter to enable the refresh loop:

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()
client.login(start_async_refresh=True)  # this will automatically hook to Jupyter's async loop to refresh the session

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID: a9689adda1614b629b6f5730ab1e129f)


This parameter essentially schedules the refresh loop in asyncio:

```Python
asyncio.create_task(self.async_refresh(), name="async_refresh")
```

Note: this parameter _only_ works when it is called in a Jupyter Notebook because it expects a running asyncio loop (which Jupyter spins up). If the method is called from a command-line script, the following error will be thrown:

```
RuntimeError: no event loop running - do not use start_async_refresh=True outside a Jupyter Notebook
```

## From Command-Line Applications

When the client is used to implement a service such as a price-streaming, order-placing, and/or portfolio-monitoring app, the application logic should include a call to refresh the tokens.

In these cases the service typically implements a `main()` function that is run asynchronously (because it handles all kinds of async tasks such as receiving websocket messages). This function should include a call to ensure that asyncio spins up the refresh loop:

In [ ]:
import asyncio

client = SaxoOpenAPIClient()
client.login()


async def main() -> None:
    # do all kinds of application logic here
    # make sure nothing in here is blocking otherwise the refresh call won't be reached

    # ensure that asyncio schedules the refresh loop
    asyncio.ensure_future(client.async_refresh())


# run the app
asyncio.run(main())

## Confirming Refreshing is Correctly Scheduled

If the refresh loop is correctly set up and the client has logging enabled (see next sample), the following DEBUG messages will come up in the logs confirming that the refresh flow is scheduled. The client waits until 30 seconds before the access token expires to refresh the session.

```
2023-03-23 15:35:32.631Z   DEBUG    client             244 async_refresh             async refresh will kick off refresh flow in 1170 seconds at: 2023-03-23 15:55:02+00:00

```